# Whisper Fine-Tuning: RTX 5090 Extreme Optimization

**Hardware Config:**
- **GPU:** RTX 5090 (32GB VRAM)
- **CPU:** 15 vCPUs
- **RAM:** 117 GB

**Optimizations:**
- `batch_size=64`: Maximizing VRAM usage for stability and speed.
- `dataloader_num_workers=15`: Utilizing all vCPUs for data loading.
- `gradient_checkpointing=False`: Disabled for speed (we have enough VRAM).
- **Fixes:** Integrated Augmentation, Dropout, Early Stopping, WandB tracking.

In [ ]:
# --- STEP 1: Install Dependencies ---
!pip install datasets transformers torchaudio evaluate jiwer torchcodec audiomentations tensorboard scikit-learn accelerate wandb
!pip install --upgrade transformers accelerate

In [ ]:
# --- STEP 2: Imports & WandB Login ---
import os
import wandb
from huggingface_hub import login

# 🔑 Log in to Weights & Biases
wandb.login()
# os.environ["WANDB_PROJECT"] = "whisper-minang-rtx5090"

In [ ]:
# --- STEP 3: Dataset Preparation (Robust) ---
import pandas as pd
import torchaudio
import torch
import numpy as np
from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift

# Config
audio_base_path = "/workspace/data/audio_train/librivox-indonesia"
metadata_path = os.path.join(audio_base_path, "metadata_train_wav.csv")
MODEL_NAME = "openai/whisper-small"
TARGET_LANGUAGE = "min"  # Target language code
SAMPLING_RATE = 16000

# 1. Load Metadata
if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"❌ Metadata file not found at {metadata_path}.")

print(f"📂 Loading metadata from: {metadata_path}")
df = pd.read_csv(metadata_path)

# 2. Filter Language (with Fallback)
if "language" not in df.columns:
    raise ValueError(f"❌ Column 'language' not found! Available: {df.columns}")

df_lang = df[df["language"] == TARGET_LANGUAGE].copy()

# Auto-fallback
if len(df_lang) == 0:
    print(f"⚠️ No samples found for '{TARGET_LANGUAGE}'. Switching to most common language.")
    most_common = df["language"].mode()[0]
    df_lang = df[df["language"] == most_common].copy()
    print(f"🔄 Switched to: '{most_common}' ({len(df_lang)} samples)")

# 3. Cleanup Paths
df_lang = df_lang.dropna(subset=["path"])
df_lang["path"] = df_lang["path"].astype(str)
df_lang["full_path"] = df_lang["path"].apply(lambda p: os.path.join(audio_base_path, p))

# Verify files exist
df_lang["exists"] = df_lang["full_path"].apply(os.path.exists)
df_lang = df_lang[df_lang["exists"]]

print(f"✅ Final Dataset Size: {len(df_lang)} samples")

# 4. Split
train_df, val_df = train_test_split(df_lang, test_size=0.1, random_state=42)
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

# 5. Processor & Mapping
print("⚙️ Loading Processor...")
# Use 'indonesian' token for better transfer learning
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language="indonesian", task="transcribe")

# 6. Augmentation Pipeline
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
])

def prepare_dataset_augmented(batch):
    # Load using torchaudio
    audio, sr = torchaudio.load(batch["full_path"])
    if sr != SAMPLING_RATE: audio = torchaudio.functional.resample(audio, sr, SAMPLING_RATE)
    if audio.shape[0] > 1: audio = audio.mean(dim=0, keepdim=True)
    
    # Apply Augmentation
    audio_np = audio.squeeze().numpy()
    augmented_audio = augment(samples=audio_np, sample_rate=SAMPLING_RATE)
    
    inputs = processor(
        audio=augmented_audio,
        sampling_rate=SAMPLING_RATE,
        text=batch["sentence"]
    )
    return {"input_features": inputs["input_features"][0], "labels": inputs["labels"]}

def prepare_dataset_clean(batch):
    # NO Augmentation for Validation
    audio, sr = torchaudio.load(batch["full_path"])
    if sr != SAMPLING_RATE: audio = torchaudio.functional.resample(audio, sr, SAMPLING_RATE)
    if audio.shape[0] > 1: audio = audio.mean(dim=0, keepdim=True)
    
    inputs = processor(
        audio=audio.squeeze().numpy(),
        sampling_rate=SAMPLING_RATE,
        text=batch["sentence"]
    )
    return {"input_features": inputs["input_features"][0], "labels": inputs["labels"]}

print("🔄 Processing dataset...")
# Use num_proc=15 (matching your vCPUs) for blazing fast preprocessing
train_ds = train_ds.map(prepare_dataset_augmented, num_proc=15)
val_ds = val_ds.map(prepare_dataset_clean, num_proc=15)

print("✅ Data ready.")

In [ ]:
# --- STEP 4: Metrics & Data Collator ---
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

@torch.no_grad()
def data_collator(features):
    input_features = [{"input_features": f["input_features"]} for f in features]
    labels = [f["labels"] for f in features]

    batch = processor.feature_extractor.pad(input_features, return_tensors="pt")
    
    # Manually pad labels
    max_len = max(len(l) for l in labels)
    padded_labels = [l + [-100]*(max_len - len(l)) for l in labels]
    batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)
    
    if (batch["labels"][:, 0] == processor.tokenizer.bos_token_id).all():
        batch["labels"] = batch["labels"][:, 1:]
        
    return batch

In [ ]:
# --- STEP 5: Model Loading & Training (RTX 5090 Power) ---
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import gc

# Cleanup memory
if 'model' in globals(): del model
if 'trainer' in globals(): del trainer
gc.collect()
torch.cuda.empty_cache()

print("🚀 Loading Model on RTX 5090...")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to("cuda")
model.config.use_cache = False

# --- ADD DROPOUT FOR REGULARIZATION ---
model.config.dropout = 0.1
model.config.attention_dropout = 0.1

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-minang-5090",
    
    # --- WANDB CONFIGURATION ---
    report_to="wandb",
    run_name="whisper-5090-run-01",
    
    # --- RTX 5090 OPTIMIZATION ---
    per_device_train_batch_size=64,   # Using 64 batch size to utilize 5090 power
    gradient_accumulation_steps=1,    # No accumulation needed with batch 64
    gradient_checkpointing=False,     # Speed over memory savings
    
    # CPU OPTIMIZATION
    dataloader_num_workers=15,        # Use all 15 vCPUs
    dataloader_pin_memory=True,       
    # -----------------------------

    learning_rate=1e-5,
    warmup_steps=20,
    max_steps=400,                    # Short run to check for overfitting first
    fp16=True,
    
    # Evaluation & Saving
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    logging_steps=10,
    predict_with_generate=True,
    generation_max_length=225,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # Stop if WER doesn't improve
)

print("✅ Configured for RTX 5090.")
print(f"   Batch Size: {training_args.per_device_train_batch_size}")
print(f"   Workers:    {training_args.dataloader_num_workers}")
print("🚀 Starting Training...")
trainer.train()

In [ ]:
# --- STEP 6: Finish Run ---
wandb.finish()